## Setup

In [5]:
#@title Downloading Libraries
#install.packages("data.table")

if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("SNPRelate")

install.packages("bootstrap")
library(bootstrap)
library(data.table)
library(utils)
library(stats)
library(SeqArray)
library(SNPRelate)
library(bootstrap)

install.packages("nlsr")
library(nlsr)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages("BiocManager"):
“installation of package ‘BiocManager’ had non-zero exit status”


ERROR: ignored

## GET CORR for 6 ancestries

In [8]:
#altered by Sasha and Jessica to include 6 ancestries 

# extract object from .RData file
get_obj <- function(Rdata){
  objname <- load(Rdata)
  if (length(objname) > 1) {
    warning(paste("Multiple objects stored in file", Rdata,
                  "\nReturning only the first object"))
  }
  return(get(objname))
}

# convert genetic distance into recombination fraction
L_to_theta <- function(cM){
  L <- cM/100
  theta <- 0.5*(1-exp(-2*L))
  return(theta)
}

# function to get distance between pairs of SNPs
get_dist <- function(s1, s2, gen.map){
  pos1 <- gen.map$cM[match(s1, gen.map$snp.id)]
  pos2 <- gen.map$cM[match(s2, gen.map$snp.id)]
  return(abs(pos2-pos1))
}

# get correlation for one pair of SNPs
get_corr <- function(s1, s2, eas, eur_was, nat, oce, sas, afr){
  # load ancestries at SNPs s1 and s2
  eas.anc <- snpgdsGetGeno(eas, snp.id = c(s1,s2), with.id = TRUE, verbose=F)
  eur_was.anc <- snpgdsGetGeno(eur_was, snp.id = c(s1,s2), with.id = TRUE, verbose=F)
  nat.anc <- snpgdsGetGeno(nat, snp.id = c(s1,s2), with.id = TRUE, verbose=F)
  oce.anc <- snpgdsGetGeno(oce, snp.id = c(s1,s2), with.id = TRUE, verbose=F)
  sas.anc <- snpgdsGetGeno(sas, snp.id = c(s1,s2), with.id = TRUE, verbose=F)
  afr.anc <- snpgdsGetGeno(afr, snp.id = c(s1,s2), with.id = TRUE, verbose=F)
  # get indices for each SNP (make sure they're in order we expected)
  ea1 <- which(eas.anc$snp.id==s1); ea2 <- which(eas.anc$snp.id == s2)
  eu1 <- which(eur_was.anc$snp.id==s1); eu2 <- which(eur_was.anc$snp.id == s2)
  n1 <- which(nat.anc$snp.id==s1); n2 <- which(nat.anc$snp.id == s2)
  o1 <- which(oce.anc$snp.id==s1); o2 <- which(oce.anc$snp.id == s2)
  sa1 <- which(sas.anc$snp.id==s1); sa2 <- which(sas.anc$snp.id == s2)
  a1 <- which(afr.anc$snp.id==s1); a2 <- which(afr.anc$snp.id == s2)

  # get correlation between all possible pairs of ancestries
  c11 <- cor(eas.anc$g[,ea1], eas.anc$g[,ea2])
  c12 <- cor(eas.anc$g[,ea1], eur_was.anc$g[,eu2])
  c13 <- cor(eas.anc$g[,ea1], nat.anc$g[,n2])
  c14 <- cor(eas.anc$g[,ea1], oce.anc$g[,o2])
  c15 <- cor(eas.anc$g[,ea1], sas.anc$g[,sa2])
  c16 <- cor(eas.anc$g[,ea1], afr.anc$g[,a2])
    
  c21 <- cor(eur_was.anc$g[,eu1], eas.anc$g[,ea2])
  c22 <- cor(eur_was.anc$g[,eu1], eur_was.anc$g[,eu2])
  c23 <- cor(eur_was.anc$g[,eu1], nat.anc$g[,n2])
  c24 <- cor(eur_was.anc$g[,eu1], oce.anc$g[,o2])
  c25 <- cor(eur_was.anc$g[,eu1], sas.anc$g[,sa2])
  c26 <- cor(eur_was.anc$g[,eu1], afr.anc$g[,a2])
    
  c31 <- cor(nat.anc$g[,n1], eas.anc$g[,ea2])
  c32 <- cor(nat.anc$g[,n1], eur_was.anc$g[,eu2])
  c33 <- cor(nat.anc$g[,n1], nat.anc$g[,n2])
  c34 <- cor(nat.anc$g[,n1], oce.anc$g[,o2])
  c35 <- cor(nat.anc$g[,n1], sas.anc$g[,sa2])
  c36 <- cor(nat.anc$g[,n1], afr.anc$g[,a2])
    
  c41 <- cor(oce.anc$g[,o1], eas.anc$g[,ea2])
  c42 <- cor(oce.anc$g[,o1], eur_was.anc$g[,eu2])
  c43 <- cor(oce.anc$g[,o1], nat.anc$g[,n2])
  c44 <- cor(oce.anc$g[,o1], oce.anc$g[,o2])
  c45 <- cor(oce.anc$g[,o1], sas.anc$g[,sa2])
  c46 <- cor(oce.anc$g[,o1], afr.anc$g[,a2])
    
  c51 <- cor(sas.anc$g[,sa1], eas.anc$g[,ea2])
  c52 <- cor(sas.anc$g[,sa1], eur_was.anc$g[,eu2])
  c53 <- cor(sas.anc$g[,sa1], nat.anc$g[,n2])
  c54 <- cor(sas.anc$g[,sa1], oce.anc$g[,o2])
  c55 <- cor(sas.anc$g[,sa1], sas.anc$g[,sa2])
  c56 <- cor(sas.anc$g[,sa1], afr.anc$g[,a2])
    
  c61 <- cor(afr.anc$g[,a1], eas.anc$g[,ea2])
  c62 <- cor(afr.anc$g[,a1], eur_was.anc$g[,eu2])
  c63 <- cor(afr.anc$g[,a1], nat.anc$g[,n2])
  c64 <- cor(afr.anc$g[,a1], oce.anc$g[,o2])
  c65 <- cor(afr.anc$g[,a1], sas.anc$g[,sa2])
  c66 <- cor(afr.anc$g[,a1], afr.anc$g[,a2])
    
    
  return(c(c11,c12,c13,c14,c15,c16,
           c21,c22,c23,c24,c25,c26,
           c31,c32,c33,c34,c35,c36,
           c41,c42,c43,c44,c45,c46,
           c51,c52,c53,c54,c55,c56,
           c61,c62,c63,c64,c65,c66))
}

In [10]:
#altered by Sasha and Jessica to include 6 ancestries 

#' Calculate Correlation in Local Ancestry (K = 3)
#'
#' Calculate the correlation of local ancestry vectors
#' for a single chromosome. Code is currently only applicable to
#' admixed populations with three ancestral populations.
#'
#' @param chrom chromosome number that you are analyzing
#' @param binsize size (in cM) of distance bins for calculating correlation; default = 0.5 cM
#' @param map map file; data frame with, at minimum, columns 'chr' and 'cM'
#' @param pop1.gds name of GDS file storing local ancestry calls for chrom of interest, with alleles coded as 1 = pop 1 ancestry, 0 = pop 2 or 3
#' @param pop2.gds name of GDS file storing local ancestry calls for chrom of interest, with alleles coded as 1 = pop 2 ancestry, 0 = pop 1 or 3
#' @param pop3.gds name of GDS file storing local ancestry calls for chrom of interest, with alleles coded as 1 = pop 3 ancestry, 0 = pop 1 or 2
#' @param verbose do you want to print updates to screen; default = TRUE
#'
#'
#' @return A data table with the observed correlation in local ancestry vectors for a subset of loci on this chromosome.
#'
#' @import data.table gdsfmt SeqArray SNPRelate
#'
#' @importFrom utils combn
#'
#' @importFrom stats cor
#'
#' @seealso \code{\link[STEAM]{get_g}} and \code{\link[STEAM]{combine_corr_chr}}
#'
#' @export
get_corr_chr <- function(chrom, binsize = 0.5, map, pop1.gds, pop2.gds, pop3.gds, pop4.gds, pop5.gds, pop6.gds, verbose = TRUE){
  ## restrict map to chrom of interest
  map.df <- subset(map, chr == chrom)

  # list all possible pairs of loci
  if(is.null(map.df$snp.id)) map.df$snp.id <- 1:(nrow(map.df))
  snps.pairs <- combn(map.df$snp.id, 2)

  # store as data table
  snps.dt <- data.table(t(snps.pairs))
  names(snps.dt) <- c('snp1','snp2')

  # add distances between pairs of SNPs
  snps.dt[, cM := get_dist(snp1,snp2,map.df)] # genetic distance bt SNPs
  snps.dt[, theta := L_to_theta(cM)] # recombination fraction bt SNPs

  # figure out which distance bin each pair is in
  snps.dt[, bin := round(cM/binsize, 0) * binsize]

  # keep 20 random pairs of SNPs per bin (or keep all pairs if < 20 in bin)
  snps.dt[, pair := paste0(snp1,'_',snp2)]
  selected <- snps.dt[, .(sample(pair, size = min(20, .N), replace = FALSE)),
                      by = .(bin)]
  snps.dt <- snps.dt[pair %in% selected$V1]
  snps.dt$pair <- NULL # remove pair info

  ## open GDS files
  pop1 <- seqOpen(pop1.gds, allow.duplicate=TRUE)
  pop2 <- seqOpen(pop2.gds, allow.duplicate=TRUE)
  pop3 <- seqOpen(pop3.gds, allow.duplicate=TRUE)
  pop4 <- seqOpen(pop4.gds, allow.duplicate=TRUE)
  pop5 <- seqOpen(pop5.gds, allow.duplicate=TRUE)
  pop6 <- seqOpen(pop6.gds, allow.duplicate=TRUE) 
      

  # set up correlation columns to store correlations
  snps.dt$corr_11 <- NA
  snps.dt$corr_12 <- NA
  snps.dt$corr_13 <- NA
  snps.dt$corr_14 <- NA
  snps.dt$corr_15 <- NA
  snps.dt$corr_16 <- NA
  snps.dt$corr_21 <- NA
  snps.dt$corr_22 <- NA
  snps.dt$corr_23 <- NA
  snps.dt$corr_24 <- NA
  snps.dt$corr_25 <- NA
  snps.dt$corr_26 <- NA
  snps.dt$corr_31 <- NA
  snps.dt$corr_32 <- NA
  snps.dt$corr_33 <- NA
  snps.dt$corr_34 <- NA
  snps.dt$corr_35 <- NA 
  snps.dt$corr_36 <- NA
  snps.dt$corr_41 <- NA
  snps.dt$corr_42 <- NA
  snps.dt$corr_43 <- NA
  snps.dt$corr_44 <- NA
  snps.dt$corr_45 <- NA  
  snps.dt$corr_46 <- NA 
  snps.dt$corr_51 <- NA
  snps.dt$corr_52 <- NA
  snps.dt$corr_53 <- NA
  snps.dt$corr_54 <- NA
  snps.dt$corr_55 <- NA
  snps.dt$corr_56 <- NA
  snps.dt$corr_61 <- NA
  snps.dt$corr_62 <- NA
  snps.dt$corr_63 <- NA
  snps.dt$corr_64 <- NA
  snps.dt$corr_65 <- NA
  snps.dt$corr_66 <- NA
      

  # loop through pairs to get correlation
  for(i in 1:nrow(snps.dt)){
    # get snp names
    snp1.i <- snps.dt$snp1[i]
    snp2.i <- snps.dt$snp2[i]
    # get correlations
    corr.i <- get_corr(snp1.i, snp2.i, eas = pop1, eur_was = pop2, nat = pop3, oce = pop4, sas = pop5, afr = pop6)
    snps.dt$corr_11[i] <- corr.i[1]
    snps.dt$corr_12[i] <- corr.i[2]
    snps.dt$corr_13[i] <- corr.i[3]
    snps.dt$corr_14[i] <- corr.i[4]
    snps.dt$corr_15[i] <- corr.i[5]
    snps.dt$corr_16[i] <- corr.i[6]
    snps.dt$corr_21[i] <- corr.i[7]
    snps.dt$corr_22[i] <- corr.i[8]
    snps.dt$corr_23[i] <- corr.i[9]
    snps.dt$corr_24[i] <- corr.i[10]
    snps.dt$corr_25[i] <- corr.i[11]
    snps.dt$corr_26[i] <- corr.i[12]
    snps.dt$corr_31[i] <- corr.i[13]
    snps.dt$corr_32[i] <- corr.i[14]
    snps.dt$corr_33[i] <- corr.i[15]
    snps.dt$corr_34[i] <- corr.i[16]
    snps.dt$corr_35[i] <- corr.i[17]
    snps.dt$corr_36[i] <- corr.i[18]
    snps.dt$corr_41[i] <- corr.i[19]
    snps.dt$corr_42[i] <- corr.i[20]
    snps.dt$corr_43[i] <- corr.i[21]
    snps.dt$corr_44[i] <- corr.i[22]
    snps.dt$corr_45[i] <- corr.i[23]
    snps.dt$corr_46[i] <- corr.i[24]
    snps.dt$corr_51[i] <- corr.i[25]
    snps.dt$corr_52[i] <- corr.i[26]
    snps.dt$corr_53[i] <- corr.i[27]
    snps.dt$corr_54[i] <- corr.i[28]
    snps.dt$corr_55[i] <- corr.i[29]
    snps.dt$corr_56[i] <- corr.i[30]
    snps.dt$corr_61[i] <- corr.i[31]
    snps.dt$corr_62[i] <- corr.i[32]
    snps.dt$corr_63[i] <- corr.i[33]
    snps.dt$corr_64[i] <- corr.i[34]
    snps.dt$corr_65[i] <- corr.i[35]
    snps.dt$corr_66[i] <- corr.i[36]
    # progress report
    if(i %% 100 == 0 & verbose == TRUE) cat('Done with rep',i,'of',nrow(snps.dt),'\n')
  }

  # close gds files
  seqClose(pop1); seqClose(pop2); seqClose(pop3); seqClose(pop4); seqClose(pop5)

  # return data table with correlation
  return(snps.dt)

}

In [12]:
#altered by Sasha and Jessica to include 6 ancestries 
example_map <- read.table(file = 'map_file.tsv', sep = '\t', header = TRUE)

# set up list to store results
corr.list <- list()

# loop through chromosomes
# NOTE: we recommend that each chromosome be analyzed separately (e.g., on distinct compute nodes) and results saved, then combined later, rather than analyzing all chromosomes in a single R session
for(i in 1:22){
  eas.gds <- paste0('gds_files_with_oce/covid_ref_chr', i, '_ancestry', 1, '_rfmix.gds')
  eur_was.gds <- paste0('gds_files_with_oce/covid_ref_chr', i, '_ancestry', 2, '_rfmix.gds')
  nat.gds <- paste0('gds_files_with_oce/covid_ref_chr', i, '_ancestry', 3, '_rfmix.gds')
  oce.gds <- paste0('gds_files_with_oce/covid_ref_chr', i, '_ancestry', 4, '_rfmix.gds')
  sas.gds <- paste0('gds_files_with_oce/covid_ref_chr', i, '_ancestry', 5, '_rfmix.gds')
  afr.gds <- paste0('gds_files_with_oce/covid_ref_chr', i, '_ancestry', 0, '_rfmix.gds')
  # run get_corr_chr
  corr.list[[i]] <- get_corr_chr(chrom = i, map = example_map, 
        pop1.gds = eas.gds, pop2.gds = eur_was.gds, pop3.gds = nat.gds, pop4.gds = oce.gds, pop5 = sas.gds, pop6=afr.gds)
    message(paste0("done with chromosome ", i))
  cat('done with chromosome', i, '\n')
}

Warning message in file(file, "rt"):
“cannot open file 'map_file.tsv': No such file or directory”


ERROR: ignored

In [13]:
#altered by Sasha and Jessica to include 6 ancestries 

#' Calculate Correlation in Local Ancestry (K = 3)
#'
#' Create single data frame with average correlation of local ancestry
#' for pairs of SNPs varying distances apart. Combines data for each chromosome
#' generated by get_corr_chr(). Code is currently applicable
#' only to admixed populations with three ancestral populations.
#'
#' @param cor.list list of data tables, one per chromosome, generated by get_corr_chr().
#'
#' @return A data table with the average correlation in local ancestry, to be used for estimating the number of generations since admixture using \code{\link[STEAM]{get_g}}.
#'
#' @import data.table
#'
#' @importFrom stats reshape
#'
#' @seealso \code{\link[STEAM]{get_g}} and \code{\link[STEAM]{get_corr_chr}}
#'
#' @export
combine_corr_chr <- function(cor.list){
  # combine into single data table
  snps.df <- rbindlist(cor.list)

  # get average within each bin
  avgs <- snps.df[,.(mean(corr_11),mean(corr_12),mean(corr_13),mean(corr_14), mean(corr_15), mean(corr_16),
                     mean(corr_21),mean(corr_22),mean(corr_23),mean(corr_24), mean(corr_25), mean(corr_26),
                     mean(corr_31),mean(corr_32),mean(corr_33),mean(corr_34), mean(corr_35), mean(corr_36),
                     mean(corr_41),mean(corr_42),mean(corr_43),mean(corr_44), mean(corr_45), mean(corr_46),
                     mean(corr_51),mean(corr_52),mean(corr_53),mean(corr_54), mean(corr_55), mean(corr_56),
                     mean(corr_61),mean(corr_62),mean(corr_63),mean(corr_64), mean(corr_65), mean(corr_66),
                     ), by = .(bin)]
  avgs[,('theta') := L_to_theta(bin)] # add recomb fraction back on

  # add col names
  names(avgs) <- c('cM',
                   'corr_11','corr_12','corr_13','corr_14', 'corr_15', 'corr_16',
                   'corr_21','corr_22','corr_23','corr_24', 'corr_25', 'corr_26',
                   'corr_31','corr_32','corr_33','corr_34', 'corr_35', 'corr_36',
                   'corr_41','corr_42','corr_43','corr_44', 'corr_45', 'corr_46',
                   'corr_51','corr_52','corr_53','corr_54', 'corr_55', 'corr_56',
                   'corr_61','corr_62','corr_63','corr_64', 'corr_65', 'corr_66',
                   'theta')

  # convert from wide to long
  avgs.long <- reshape(avgs, direction = 'long', varying = list(2:26),
                       v.names = 'corr', idvar = 'bin', timevar = 'anc',
                       times = c('1_1','1_2','1_3','1_4','1_5','1_6',
                                 '2_1','2_2','2_3','2_4','2_5','2_6',
                                 '3_1','3_2','3_3','3_4','3_5','3_6',
                                 '4_1','4_2','4_3','4_4','4_5','4_6',
                                 '5_1','5_2','5_3','5_4','5_5','5_6',
                                 '6_1','6_2','6_3','6_4','6_5','6_6'))

  avgs.long$anc2 <- avgs.long$anc
  avgs.long$anc2[avgs.long$anc2=='2_1'] <- '1_2'
  avgs.long$anc2[avgs.long$anc2=='3_1'] <- '1_3'
  avgs.long$anc2[avgs.long$anc2=='3_2'] <- '2_3'
  avgs.long$anc2[avgs.long$anc2=='4_1'] <- '1_4'
  avgs.long$anc2[avgs.long$anc2=='4_2'] <- '2_4'
  avgs.long$anc2[avgs.long$anc2=='4_3'] <- '3_4'  
  avgs.long$anc2[avgs.long$anc2=='5_1'] <- '1_5'
  avgs.long$anc2[avgs.long$anc2=='5_2'] <- '2_5' 
  avgs.long$anc2[avgs.long$anc2=='5_3'] <- '3_5'
  avgs.long$anc2[avgs.long$anc2=='5_4'] <- '4_5'
  avgs.long$anc2[avgs.long$anc2=='6_1'] <- '1_6'  
  avgs.long$anc2[avgs.long$anc2=='6_2'] <- '2_6'  
  avgs.long$anc2[avgs.long$anc2=='6_3'] <- '3_6'    
  avgs.long$anc2[avgs.long$anc2=='6_4'] <- '4_6' 
  avgs.long$anc2[avgs.long$anc2=='6_5'] <- '5_6' 

  # save final results as data frame
  lacorr.df <- as.data.frame(avgs.long)
  lacorr.df <- lacorr.df[,c('theta','corr','anc2')]
  names(lacorr.df) <- c('theta','corr','anc')
  return(lacorr.df)
}

In [14]:
corr_K3 <- combine_corr_chr(corr.list)

ERROR: ignored

In [15]:
corr_K3

ERROR: ignored

##RUNNING NLS

In [ ]:
#altered by Sasha and Jessica (NLXB)

#' Run Unconstrained NLS Regression
#'
#' Estimate the number of generations since admixture (g) from the
#' observed local ancestry correlation curves using non-linear
#' least squares regression. More details in Grinde et al. (TBD).
#'
#' @param lacorr local ancestry correlation; data frame with 3 columns: theta = recomb frac; corr = correlation; anc = ancestry components (e.g., 1_1, 1_2, 1_3, 2_2, 2_3, 3_3)
#' @param k1,k2 ancestry component indices (number between 1 and K)
#' @param start.a starting value for intercept; default = 0
#' @param start.b starting value for slope; default = 1
#' @param start.g starting value for g; default = 10
#'
#' @return A single number indicating the estimated number of generations since admixture.
#'
#' @examples
#' run_unc_nls(lacorr = example_corr, k1 = 1, k2 = 1)
#' run_unc_nls(lacorr = example_corr, k1 = 1, k2 = 2)
#'
#' @importFrom stats nls
#'
#' @export
run_unc_nls <- function(lacorr, k1, k2, start.a = 0, start.b = 1, start.g = 10){
  # get name of ancestry pair
  a.pair <- paste(sort(c(k1,k2),decreasing=F),collapse='_')
  # run NLS
  mod <- eval(nlxb(corr ~ a + b*(1-theta)^g,
                                data = subset(lacorr,anc==a.pair), 
                                start = c(a = start.a, b = start.b, g = start.g),
                                lower = c(a = -Inf, b = -Inf, g = 2),
                                upper = c(a = Inf, b = Inf, g = 25)))
                                #control = list(maxiter = 10000, tolerance = 1e-5, minFactor = 0)))

  # return model
  return(mod)
}

In [ ]:
#altered by Sasha and Jessica (NLXB)

#' Estimate Number of Generations since Admixture
#'
#' Estimate the number of generations since admixture (g) from the
#' observed local ancestry correlation curves using non-linear
#' least squares regression. More details in Grinde et al. (TBD).
#'
#' @param lacorr local ancestry correlation; data frame with 3 columns: theta = recomb frac; corr = correlation; anc = ancestry components (e.g., 1_1, 1_2, 1_3, 2_2, 2_3, 3_3)
#' @param start.a starting value for intercept; default = 0
#' @param start.b starting value for slope; default = 1
#' @param start.g starting value for g; default = 10
#'
#' @return A single number indicating the estimated number of generations since admixture.
#'
#' @examples
#' get_g(lacorr = example_corr)
#' get_g(lacorr = example_corr_K3)
#'
#' @importFrom stats nls coef
#'
#' @export
get_g <- function(lacorr, start.a = 0, start.b = 1, start.g = 10){
    # get ancestry pairs from lacorr
    anc.pairs.vec <- as.character(unique(lacorr$anc))
    anc.pairs.list <- strsplit(anc.pairs.vec,'_')
    ap1 <- unlist(lapply(anc.pairs.list,function(x) x[1]))
    ap2 <- unlist(lapply(anc.pairs.list,function(x) x[2]))
    anc.pairs <- data.frame(X1=ap1,X2=ap2)
    # run on each ancestry pair to get individual slopes/intercepts
    ab <- list()
    for(i in 1:nrow(anc.pairs)){
      a1 <- anc.pairs[i,1]; a2 <- anc.pairs[i,2]
      ab[[i]] <- coef(run_unc_nls(lacorr,a1,a2,start.a,start.b,start.g))[1:2]
    }
    names(ab) <- paste(anc.pairs[,1],anc.pairs[,2],sep='_')
    # add newly estimated constants to data frame
    true_ab <- sapply(lacorr$anc, function(x) ab[[x]])
    lacorr$a <- true_ab[1,]
    lacorr$b <- true_ab[2,]
    # run NLS
    mod <- eval(nlxb(corr ~ a + b*(1-theta)^g,
                                data = lacorr, 
                                start = c(g = start.g),
                                lower = c(g = 2),
                                upper = c(g = 25)))
    # return g
    g <- coef(mod)
    return(g)
}

example get corr

In [ ]:
load("corr_K6_0.rda") #african

In [ ]:
load("corr_K6_1.rda") #EastAsian

In [ ]:
load("corr_K6_2.rda") #European

In [ ]:
load("corr_K6_5.rda")

In [ ]:
get_g(corr_K6_0) #african
get_g(corr_K6_1) #EastAsian
get_g(corr_K6_2) #European
get_g(corr_K6_3) #NATIVE AMERICAN 
get_g(corr_K6_4) #OCEANIC
get_g(corr_K6_5) #SOUTH ASIAN

g 
11.85447 
attr(,"pkgname")
[1] "nlsr"

g 
12.35601 
attr(,"pkgname")
[1] "nlsr"

g 
10.77205 
attr(,"pkgname")
[1] "nlsr"

g 
11.57381 
attr(,"pkgname")
[1] "nlsr"

g 
14.68369 
attr(,"pkgname")
[1] "nlsr"

g 
12.93741 
attr(,"pkgname")
[1] "nlsr"

In [ ]:
ancestry_df = read.csv("ADMIXTURE_ancestries_fractions_1_.tsv", header = TRUE, sep = "\t", dec = ".")
cleaned_df <- ancestry_df[-c(1, 2, 8)]
cleaned_df[nrow(ancestry_df) + 1,] = c(0, 0, 0, 0, 1, 0)
cleaned_df = cleaned_df[,c(1,6,2,5,4,3)]
example_props <- cleaned_df

In [ ]:
example_props <- cleaned_df

##Helper Functions

In [ ]:
#' Pairwise distances between markers
#'
#' Helper function for test statistic simulation.
#' Get distances between consecutive markers.
#'
#' @param map data frame with m rows and 2 columns ('chr' containing chromosome number and 'cM' containing genetic position in centimorgans), where m = no. markers
#'
#' @return Numeric vector (length m-1) of distances between markers.
get_deltas <- function(map){
  # check for missing values
  if(sum(is.na(map$cM))>0) stop('Missing values in cM column')
  if(sum(is.na(map$chr))>0) stop('Missing values in chr column')

  # initial distance between adjacent markers
  deltas <- diff(map$cM)

  # set distance between chromosomes to 100000 (infty)
  chr_starts <- sapply(1:22, function(x) which(map$chr==x)[1])
  deltas[chr_starts-1] <- 100000

  # return deltas
  return(deltas)
}

#' Helper Function: Calculate Scalars
#'
#' Helper function for test statistic simulation.
#' Calculate scalars needed for test stat simulation.
#'
#' @param deltas vector of distances between consecutive markers
#' @param g the generations since admixture.
#'
#' @return List of two vectors with two sets of scalars.
get_ab <- function(deltas,g){
  ## calculate a_{i,j}, b_{i,j} from deltas
  beta <- 0.01*g
  a <- exp(-beta*deltas)
  b <- sqrt(1-exp(-2*beta*deltas))
  return(list(a=a,b=b))
}

#' Expected correlation: one locus, square root
#'
#' Helper function for test statistic simulation.
#' Square root of expected correlation matrix at one locus.
#'
#' @param ep vector of length K (K = no. ancestral pops) containg expected/average admixture proportions
#'
#' @return Matrix with K rows and K-1 columns.
get_L <- function(ep){
  # number of ancestral pops
  K <- length(ep)
  # get correlation matrix
  sig <- get_sigma(ep=ep)
  # get square root
  if(K==2){
    # hardcode
    L <- matrix(c(1,-1),nrow=2)
  } else if(K==3){
    # hardcode
    L <- matrix(c(1,0,
                  sig[1,2],sqrt(1-sig[1,2]^2),
                  sig[1,3],-sqrt(1-sig[1,3]^2)),
                byrow=T,nrow=3)
  } else{
    # check for mgcv package
    if(requireNamespace("mgcv", quietly = TRUE)){
      # mroot
      L <- mgcv::mroot(sig,rank=K-1,method='chol')
    } else{
      L <- NULL
      cat('Please install package \"mgcv\".')
    }
  }
  return(L)
}

#' Expected correlation: one locus, all ancestries
#'
#' Helper function for test statistic simulation.
#' Calculate expected correlation at single locus
#' for all pairs of ancestries.
#'
#' @param ep vector of length K (K = no. ancestral pops) containg expected/average admixture proportions
#'
#' @return Correlation matrix.
get_sigma <- function(ep){
  # get offdiagonal elements
  sigma <- outer(ep,ep,get_sigma_ij)
  # replace diagonal elements with 1
  diag(sigma) <- 1
  # return correlation matrix
  return(sigma)
}

#' Expected correlation: one locus, one pair of ancestries
#'
#' Helper function for test statistic simulation.
#' Calculate expected correlation at single locus
#' for a single pair of ancestries.
#'
#' @param ep1,ep2 expected (average) admixture proportion for two ancestral populations
#'
#' @return Single number (the correlation).
get_sigma_ij <- function(ep1,ep2){
  -ep1*ep2/sqrt(ep1*(1-ep1)*ep2*(1-ep2))
}

#' Helper Function: Calculate Quantile
#'
#' Helper function for test statistic simulation.
#' Get upper alpha quantile of vector.
#'
#' @param t vector of test statistics
#' @param alpha the level for family-wise error rate control; default is 0.05.
#'
#' @return Single number.
upper_alpha <- function(t,alpha){
  ## Get upper alpha percentile
  quantile(t,prob=1-alpha)
}

In [ ]:
 #' Simulate Admixture Mapping Test Statistics
#'
#' Simulate admixture mapping test statistics (one time)
#' from their joint asymptotic distribution under the null
#' (see Grinde et al., 2018).
#'
#' @param m the number of markers.
#' @param K the number of ancestral populations
#' @param as vector of scalars (length m) based on generations since admixture and distance between markers.
#' @param bs vector of scalars (length m) based on generations since admixture and distance between markers.
#' @param L matrix (K x K-1) that is square root of correlation matrix at single locus
#'
#' @return A single nummber indicating the largest simulated test statistic.
#'
#' @examples
#' simstat_once(m = 2, K = 2, as = example_ab$a, bs = example_ab$b, L = example_L)
#'
#' @importFrom stats rnorm
#'
#' @export
simstat_once <- function(m,K,as,bs,L){
  # initialize max
  Zmax <- 0

  # generate all random normals that we'll need
  norms <- rnorm(m*(K-1),0,1)

  # generate indices for grabbing normals
  starts <- seq(from=1, to=(K-1)*m, by=K-1)
  ends <- seq(from=K-1, to=(K-1)*m, by=K-1)

  # get stats at first marker
  Zlast <- L %*% norms[(starts[1]):(ends[1])]

  # loop through remaining markers
  for(i in 2:m){
    # generate stats at marker i
    Znew <- as[i] * Zlast +
      bs[i] * L %*% norms[(starts[i]):(ends[i])]
    # check if we have a new max
    if(max(abs(Znew)) > Zmax) Zmax <- max(abs(Znew))
    # update Zlast for next marker
    Zlast <- Znew
  }

  # return max
  return(Zmax)
}

In [ ]:
#' Significance Threshold Estimation for Admixture Mapping using Test Statistic Simulation
#'
#' Compute genome-wide significance (test statistic or p-value) threshold for admixture
#' mapping by simulating admixture mapping test statistics from their joint asymptotic
#' distribution under the null (see Grinde et al., 2018) and finding
#' the threshold that controls the empirical family-wise error rate.
#'
#' @param g the generations since admixture.
#' @param map data frame with m rows and at least 2 columns ('chr' containing chromosome number and 'cM' containing genetic position in centimorgans), where m = no. markers
#' @param props data frame (n x K) of admixture proportions, where n = no. individuals and K = no. ancestral populations
#' @param nreps the number of repetitions for the simulation study; default is 10000.
#' @param alpha the level for family-wise error rate control; default is 0.05.
#' @param type the type of threshold that should be returned: \code{"stat"} for test statistic or \code{"pval"} for p-value; defaults to pval.
#'
#' @return A single number indicating the estimated significance threshold (either test statistic or p-value).
#'
#' @examples
#' get_thresh_simstat(g = 6, map = example_map, props = example_props, nreps = 10)
#' get_thresh_simstat(g = 6, map = example_map, props = example_props, nreps = 10, type="stat")
#'
#' @importFrom stats quantile pnorm
#'
#' @export

get_thresh_simstat <- function(g, map, props, nreps=10000, alpha=0.05, type="pval"){
  # get distances between adjacent markers
  dlt <- c(0,get_deltas(map)) # length m

  # pre-calculate constants for test stat sim (a's and b's)
  ab <- get_ab(dlt,g)

  # get average admixture proportions
  avg_props <- apply(props,2,mean,na.rm=T)

  # calculate the matrix L
  L <- get_L(avg_props) # could condense with calculating avg

  # simulate test stats nreps times
  max_stats <- replicate(nreps, simstat_once(m = nrow(map), K = ncol(props), as = ab$a, bs = ab$b, L = L))

  # get upper alpha quantile
  zstar <- upper_alpha(max_stats, alpha)

  # get 95% bootstrap CI for threshold (5k reps)
  if (requireNamespace("bootstrap", quietly = TRUE)) {
    z_ci <- quantile(bootstrap::bootstrap(max_stats, nboot = 5000, theta = upper_alpha, alpha)$thetastar, c(0.025,0.975))
  } else{
    z_ci <-c(NA,NA)
    cat('Install package \"bootstrap\" to get confidence interval. \n')
  }

  # return threshold
  if(type == "stat"){
    thresh <- zstar
    thresh_ci <- z_ci
  } else if(type == "pval"){
    thresh <- 2 * pnorm(zstar, lower.tail = F)
    thresh_ci <- 2 * pnorm(z_ci, lower.tail = F)
  } else{
    cat("Please specify type = 'stat' or type = 'pval' \n")
  }
  return(list(threshold = thresh, ci = thresh_ci))
}

In [ ]:
example_map <- read.table(file = 'map_file.tsv', sep = '\t', header = TRUE)

In [ ]:
g: 11.8544748989489
g: 12.3560136446421
g: 10.7720488583112

ERROR: ignored

In [ ]:
get_thresh_simstat(g = 11.3568297666691, map = example_map, props = example_props)

[1] 5.794256